In [1]:
# Image compression for multi images by HOSVD 
# 2017,2018 Tsuyoshi Okubo
# 2019, October, modified by TO

By using the low rank approximation through HOSVD, perform data compression of multiple gray scale images. 

You can change sample images by modifying file opens.
We assume filenames are "1.jpg", "2.jpg", ...
Please set proper value for "n_image" depending the number of images.

Also, you can set the rank of approximation by varying "chi" and "chi_p", which correspond the rank for images and the rank for # of images.

The output images (after data compression) are saved into "outputs/" directory.

Let's see, how the images change when you change the ranks.

In [2]:
## import libraries
from PIL import Image ## Python Imaging Library
import numpy as np ## numpy

In [3]:
n_image = 10 ## set number of images

In [4]:
array=[]
for i in range(1,n_image+1):
    img = Image.open("./samples/"+repr(i)+".bmp") ## load bmp image    
    #img = Image.open("./samples/"+repr(i)+".jpg") ## load jpg image    
    array.append(np.array(img,dtype=float)) ## put into ndarray
    ## array.append(np.array(img.convert("L",dtype=float))) ## put into ndarray (for color images)
array=np.array(array).transpose(1,2,0) 
print("Array shape:" +repr(array.shape)) ## print array shape

Array shape:(112, 92, 10)


In [5]:
array_truncated = np.zeros(array.shape)

chi = 30 ## rank for images
chi_p = 9 ## rank for # of images
print("HOSVD: chi=" +repr(chi) + ", chi_p=" + repr(chi_p))

HOSVD: chi=30, chi_p=9


In [6]:
## row
matrix = np.reshape(array,(array.shape[0],array.shape[1]*array.shape[2]))
u,s,vt = np.linalg.svd(matrix[:,:],full_matrices=False) ## svd 
    
#truncation
u1 = u[:,:chi]

## column
matrix = np.reshape(np.transpose(array,(1,0,2)),(array.shape[1],array.shape[0]*array.shape[2]))
u,s,vt = np.linalg.svd(matrix[:,:],full_matrices=False) ## svd 
    
#truncation
u2 = u[:,:chi]

## layer
matrix = np.reshape(np.transpose(array,(2,0,1)),(array.shape[2],array.shape[0]*array.shape[1]))
u,s,vt = np.linalg.svd(matrix[:,:],full_matrices=False) ## svd 
    
#truncation
u3 = u[:,:chi_p]

In [7]:
## make projectors
p1 = np.dot(u1,(u1.conj()).T)
p2 = np.dot(u2,(u2.conj()).T)
p3 = np.dot(u3,(u3.conj()).T)

In [8]:
## make truncated array
array_truncated = np.tensordot(np.tensordot(np.tensordot(array,p1,axes=(0,1)),p2,axes=(0,1)),p3,axes=(0,1))
normalized_distance = np.sqrt(np.sum((array-array_truncated)**2))/np.sqrt(np.sum(array**2))
print("Low rank approximation by HOSVD with chi= " +repr(chi)+ ", chi_p= "+repr(chi_p))
print("Normalized distance:" +repr(normalized_distance)) ## print normalized distance

Low rank approximation by HOSVD with chi= 30, chi_p= 9
Normalized distance:0.059601740214741024


In [9]:
for i in range(1,n_image):    
    img_truncated = Image.fromarray(np.uint8(np.clip(array_truncated[:,:,i-1],0,255))) ## convert to each image
    img_truncated.save("./outputs/"+repr(i)+".bmp") ## save compressed image
    ##img_truncated.save("./outputs/"+repr(i)+".png") ## save compressed image